In [1]:
'''
找到文章中的grammar
'''
import jieba
import re
jieba.setLogLevel(jieba.logging.INFO)

# 句子分词
def cut_sentence(sentence):
    # 使用jieba切分
    seg_list = jieba.cut(sentence, cut_all=False)
    cut_word_list = []
    for word in seg_list:
        cut_word_list.append(word)
    return cut_word_list

word_patterns = {
    "1":"^([上下里外前后左右东南西北]|上边|下边|里边|外边|前边|后边|左边|右边|东边|南边|西边|北边)$",
    "2":"^[会能]$",
    "3":"^[想要]$",
    "4":"^(多|多少|几|哪|哪儿|哪里|哪些|什么|谁|怎么)$",
    "5":"^([我你您他她]|我们|你们|他们|她们)$",
    "6":"^(这|那|这儿|那儿|这里|那里|这些|那些|别的|有的)$",
    "7":"^[一二两三四五六七八九零十百半]$",
    "8":"^[杯本个家间口块页]$",
    "9":"^(非常|[很太真最])$",
    "10":"^(都|一块儿|一起)$",
    "11":"^(马上|先|有时|在|正|正在)$",
    "12":"^(常|常常|再)$",
    "13":"^[还也]$",
    "14":"^(别|不|没|没有)$",
    "15":"^从$",
    "16":"^在$",
    "17":"^[跟和]$",
    "18":"^比$",
    "19":"^(跟|还是|和)$",
    "20":"^[的地]$",
    "21":"^了$",
    "22": "^[吧了吗呢]$",
    "49": "^(可能|可以)$",
    "50": "^(该|应该)$",
    "51": "^愿意$",
    "53": "^(多久|为什么|怎么样|怎样)$",
    "54": "^(别人|大家|它|它们|咱|咱们|自己)$",
    "55": "^(那么|那样|这么|这样)$",
    "56": "^([一-龥])\\1$|^([一-龥])\\1([一-龥])\\2$",
    "57": "^[千万亿]$",
    "58": "^[层封件条位]$",
    "59": "^[遍次场回下]$",
    "60": "^(分钟|年|天|周)$",
    "61": "^(多|多么|好|更|十分|特别|挺|有一*点儿)$",
    "62": "^(全|一共|只)$",
    "63": "^(刚|刚刚|还|忽然|一直|已经)$",
    "64": "^(重新|经常|老|老是|又)$",
    "65": "^就$",
    "66": "^故意$",
    "67": "^(必须|差不多|好像|也许)$",
    "68": "^(才|都|就|正好)$",
    "69": "^当$",
    "70": "^往$",
    "71": "^向$",
    "72": "^从$",
    "73": "^对$",
    "74": "^给$",
    "75": "^离$",
    "76": "^为$",
    "77": "^(或|或者)$",
    "78": "^(不过|但|但是|而且|那|如果|虽然|只要)$",
    "79": "^得$",
    "80": "^过$",
    "81": "^着$",
    "82": "^(啊|吧|的)$",
    "83": "^(的话|等)$",
    "84": "^喂$",
    "133": "^敢$",
    "134": "^需要$",
    "135": "^(帮忙|点头|放假|干杯|见面|结婚|看病|睡觉|洗澡|理发|说话)$",
    "136": "^(打开|看见|离开|完成)$",
    "138": "^(各|各位|各种|每|任何)$",
    "139": "^(把|行|架|群|束|双|台|张|支|只|种)$",
    "140": "^(顿|口|眼)$",
    "142": "^(比较|更加|还|相当)$",
    "143": "^(光|仅|仅仅|就|至少)$",
    "144": "^(本来|才|曾经|从来|赶紧|赶快|立刻|连忙|始终|已|早已)$",
    "145": "^(通常|往往|总|总是)$",
    "146": "^再$",
    "147": "^(互相|尽量|亲自|相互)$",
    "148": "^(大概|恐怕)$",
    "149": "^(白|并|当然|到底|反正|根本|果然|简直|绝对|难道|其实|千万|确实|只好|终于)$",
    "150": "^由$",
    "151": "^自从$",
    "152": "^朝$",
    "153": "^为$",
    "154": "^向$",
    "155": "^(由于|因为)$",
    "156": "^为了$",
    "157": "^(把|被|叫|让)$",
    "158": "^除了$",
    "159": "^(按|按照)$",
    "160": "^(并且|不光|不仅|另外|要是|因此|由于|只有)$",
    "161": "^哈哈$",
    "212": "^得$",
    "213": "^人家$",
    "214": "^(打|袋|根|卷|棵|批)$",
    "215": "^(碗|脸|手|屋子|桌子|刀|针)$",
    "216": "^(格外|极|极其)$",
    "217": "^共$",
    "218": "^(按时|即将|急忙|渐渐|尽快)$",
    "219": "^(一再|再三)$",
    "220": "^却$",
    "221": "^未必$",
    "222": "^(几乎|似乎)$",
    "223": "^(的确|反而|还|竟然|究竟)$",
    "224": "^自$",
    "225": "^对于$",
    "226": "^关于$",
    "227": "^替$",
    "228": "^根据$",
    "229": "^作为$",
    "230": "^(并|以及)$",
    "231": "^(此外|而|而是|既然|可见|甚至|假如|总之)$",
    "232": "^似的$",
    "233": "^啊$",
    "288": "^(彼此|如此)$",
    "289": "^(册|朵|幅|届|颗|匹|扇)$",
    "290": "^(过于|可|稍|稍微|尤其)$",
    "291": "^大都$",
    "292": "^(不时|将|将要|仍旧|时常|时刻|依旧|一向)$",
    "293": "^(偶尔|再次)$",
    "294": "^偷偷$",
    "295": "^(毕竟|不免|差一*点儿|倒是|干脆|就|居然|可|明明|总算)$",
    "296": "^随着$",
    "297": "^将$",
    "298": "^由$",
    "299": "^凭$",
    "300": "^依据$",
    "301": "^依照$",
    "302": "^(从而|加上|完了|一旦)$",
    "303": "^也好$",
    "361": "^(本|此)$",
    "362": "^(餐|串|滴|副|股|集|枝)$",
    "363": "^(番|声|趟)$",
    "364": "^(特|异常)$",
    "365": "^(尽|净|一齐|一同)$",
    "366": "^(时时|一时|早晚)$",
    "367": "^便$",
    "368": "^(不禁|赶忙|亲眼|特地|特意)$",
    "369": "^仿佛$",
    "370": "^(才|刚好|偏|恰好)$",
    "371": "^于$",
    "372": "^沿着*$",
    "373": "^(同|与)$",
    "374": "^至于$",
    "375": "^因$",
    "376": "^除$",
    "377": "^据$",
    "378": "^(而|同|与)$",
    "379": "^(不料|可|若)$",
    "380": "^所$",
    "381": "^(罢了|啦|嘛)$",
}
sent_patterns = {
    "93": "还是.+吧",
    "94": "又.+又.+",
    "95": "在*.+(以前|以后|前|后)",
    "115": "只要.+，就.+",
    "117": "一.+就.+",
    "129": "该.+了",
    "130": "要.+了|快要.+了|就要.+了",
    "164": "不[一-龥]不[一-龥]",
    "165": "看起来",
    "166": "看上去",
    "167": "有的是",
    "168": "除了.+(以外)*，.+(还|也|都).+",
    "169": "从.+起",
    "170": "对.+来说",
    "171": "一.+(也|都)(不|没).+",
    "172": "越.+越.+",
    "190": "也*.+，也.+",
    "191": "一会儿.+，一会儿.+",
    "192": "一方面.+，另一方面.+",
    "193": "又.+，又.+",
    "194": "首先.+，然后.+",
    "195": ".+，并且.+",
    "196": "(不仅|不光).+，(还|而且).+",
    "197": "不是.+，就是.+",
    "198": ".+([一-龥]+)是\\1，(就是|不过).+",
    "199": "要是.+，就.+",
    "200": "只有.+，才.+",
    "201": "(由于)*.+，(所以|因此).+",
    "202": "为了.+，.+",
    "203": ".+了.+就*.+",
    "234": "大[一-龥]大[一-龥]",
    "235": "一[一-龥]一[一-龥]",
    "236": "看来",
    "237": "来得及|来不及",
    "238": "说不定",
    "239": "一般来说",
    "241": "自*.+以来",
    "242": "由.+组成",
    "243": "在.+方面",
    "244": "在.+(上|下|中)",
    "254": "不是.+，而是.+",
    "255": "既.+，(又|也).+",
    "256": "首先.+，其次.+",
    "257": ".+，于是.+",
    "258": ".+，甚至.+",
    "259": "或者.+，或者.+",
    "260": ".+，然而.+",
    "261": ".+，否则.+",
    "262": "假如.+，就*.+",
    "263": "万一.+，就*.+",
    "264": "不管.+，(都|也).+",
    "265": "无论.+，(都|也).+",
    "266": "既然.+，就.+",
    "267": ".+，可见.+",
    "268": "哪怕.+，(也|还).+",
    "269": ".+，好.+",
    "271": "不.+也.+",
    "278": "不([一-龥]{1,2})白不\\1",
    "280": "没*有什么好*[一-龥]{1,2}的",
    "281": "([一-龥]{1,2})是\\1，([一-龥]{1,2})是\\2",
    "282": "([一-龥]{1,2})也得\\1，不\\1也得\\1",
    "283": "[一-龥]{1,2}就是了",
    "284": "还[一-龥]{1,2}呢",
    "285": "你[一-龥]{1,2}你的吧",
    "286": "(让|叫)你([一-龥]{1,2})你就\\1",
    "287": "(说什么|怎么着*)也得[一-龥]+",
    "304": "([一-龥])来\\1去",
    "305": "([一-龥])着\\1着",
    "306": "没[一-龥]没[一-龥]",
    "307": "说([一-龥])就\\1",
    "308": "有[一-龥]有[一-龥]",
    "309": "不得了",
    "310": "不敢当",
    "311": "得了",
    "312": "用不着",
    "313": "从.+来看",
    "314": "到.+为止",
    "315": "够.+的",
    "316": "拿.+来说",
    "317": "([一-龥])的\\1，([一-龥])的\\2",
    "318": "在.+看来",
    "331": "或是.+，或是.+",
    "332": "尽管.+，(但是|可是).+",
    "333": "一旦.+，就.+",
    "334": "要是.+，就*.+，否则.+",
    "335": "除非.+，才.+",
    "336": "除非.+，(否则|不然).+",
    "337": ".+，因而.+",
    "338": "即使.+，也.+",
    "339": ".+，为的是.+",
    "340": ".+，以便.+",
    "341": "没有.+就没有.+",
    "342": "再.+也.+",
    "347": "[一-龥]{1,2}也不是，[一-龥]{1,2}也不是",
    "348": "([一-龥]{1,2})也\\1不得，([一-龥]{1,2})也\\2不得",
    "349": "[一-龥]+是它，[一-龥]+也是它",
    "350": "([一-龥]{1,2})着也是\\1着",
    "351": "([一-龥]+)归\\1，([一-龥]+)归\\2",
    "352": "不管怎样说",
    "353": "看你[一-龥]{1,2}的|瞧他[一-龥]{1,2}的",
    "354": "真有(你|他|她)的",
    "355": "([一-龥]{1,2})什么\\1",
    "356": "什么([一-龥]{1,2})不\\1的*",
    "383": "或[一-龥]或[一-龥]",
    "384": "无[一-龥]无[一-龥]",
    "385": "[一-龥]这[一-龥]那",
    "386": "左[一-龥]右[一-龥]",
    "387": "不怎么",
    "388": "不怎么样",
    "389": "好不*容易",
    "390": "那倒也*是",
    "391": "就是说|这就是说",
    "392": "算了",
    "394": "东一([一-龥]+)，西一\\1",
    "395": "为了.+而.+",
    "400": "时而.+，时而.+",
    "401": "一时.+一时.+",
    "402": ".+便.+",
    "403": "(不但不|不但没有).+，反而.+",
    "404": "不是.+，(还|还是).+",
    "405": "连.+(都|也).+，.+更.+",
    "406": "要么.+，要么.+",
    "407": "虽.+，(但|可|却|也).+",
    "408": ".+，(要不然|不然).+",
    "409": "凡是.+，都.+",
    "410": "(就算|就是).+也.+",
    "411": "不.+不.+",
    "414": "[一-龥]{1,2}到[一-龥]{1,2}头上来了",
    "415": "([一-龥]+)就\\1吧",
    "416": "([一-龥]+)是\\1",
    "417": "不([一-龥]+)不.+，一\\1.+",
    "418": "好你个[一-龥]{1,2}",
    "420": "早也*不([一-龥]+)，晚也*不\\1",
    "422": "放着[一-龥]{1,2}不[一-龥]{1,2}",
    "423": "([一-龥]+)来\\1去，(都是|就是).+",
    "424": "([一-龥]+)了就\\1了，没*有.+",
    "425": "这也不[一-龥]{1,2}，那也不[一-龥]{1,2}|那也不[一-龥]{1,2}，这也不[一-龥]{1,2}",
    
}

# 计算文章中的new_level语法分布
def count_sentence_grammar_level_new_word(text):
    '''
    输出文本，输出文本中的语法匹配
    :param text: 一个文本
    :return: label语法标号；match语法点
    '''
    # 文本清洗+文本分词
    text_cut = cut_sentence(text)

    label = [] # 匹配的语法标号
    match = [] # 在句中匹配的语法

    # 找出单词层级的语法
    for word in text_cut:
        for i in word_patterns:
            if (re.search(word_patterns[i], word)):
                label.append(int(i))
                match.append(re.search(word_patterns[i], word).group())
                
    return label, match

# 计算文章中的new_level语法分布
def count_sentence_grammar_level_new_sent(text):
    '''
    输出文本，输出文本中的语法匹配
    :param text: 一个文本
    :return: label语法标号；match语法点
    '''
    # 文本清洗+文本分词
    text_cut = cut_sentence(text)

    label = [] # 匹配的语法标号
    match = [] # 在句中匹配的语法

    # 找出句子层级的语法
    for i in sent_patterns:
        if (re.search(sent_patterns[i], text)):
            label.append(int(i))
            match.append(re.search(sent_patterns[i], text).group())

    return label, match

#将长句按逗号分段，便于词级语法匹配更精准
def short_cut(sentence):
    flagchs = ['，','！','？','：']
    shortse = []
    start = -1
    end = 0
    for i in range(0,len(sentence)):
        for flagch in flagchs:
            if sentence[i] == flagch:
                end = i
                if end-start >=2:
                    se = sentence[start+1:end]
                    shortse.append(se)
                start = i
    return shortse

In [2]:
import numpy as np
import zhon
import pandas as pd
from pandas import DataFrame

data2012 = pd.read_excel('data/2010.xlsx')
standard = pd.read_excel('标准.xlsx')

sentences = []
labels = []
matches = []
for t in data2012['text']:
    #先匹配词级语法点
    shorterwords = short_cut(t)
    for shorterword in shorterwords:
        label, match = count_sentence_grammar_level_new_word(shorterword)
        i = 0
        for labelid in label:
            labelname = standard.loc[labelid-1,'grammar']
            sentences.append(shorterword)
            labels.append(labelname)
            matches.append(match[i])
            #print(shorterword,labelid,labelname,match[i])
            i+=1
    #再匹配句级语法点       
    label, match = count_sentence_grammar_level_new_sent(t)
    i=0
    for labelid in label:
        labelname = standard.loc[labelid-1,'grammar']
        labelcontent = standard.loc[labelid-1,'content']
        sentences.append(match[i])
        labels.append(labelname)
        matches.append(labelcontent)
        #print(cut,match[i],labelname,labelcontent)
        i+=1
print(sentences[0],labels[0],matches[0])

这种植物在这个季节长得很快 词类副词时间副词 在


In [3]:
df = pd.DataFrame({'text':sentences, 'label':labels, 'match':matches})
df.to_csv('result2010.csv',encoding='utf-8')